In [169]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
#!pip install lime
import lime
import lime.lime_tabular
import shap
import pandas as pd

In [1]:
dataset = datasets.load_iris()

In [2]:
X, y = dataset.data, dataset.target

In [170]:
df = pd.read_csv("classification_set.csv")
df.head(), df.columns

(   Unnamed: 0  Informative Feature 1  Informative Feature 2  \
 0           0               1.193633               1.663061   
 1           1               1.048270               1.169108   
 2           2               1.009388               1.361849   
 3           3              -0.140479               0.782680   
 4           4               1.383343               2.188621   
 
    Informative Feature 3  Redundant Feature 1  Nuisance Feature 1  \
 0              -0.275160             0.530091            0.426154   
 1               0.052607             0.000069           -0.365590   
 2              -0.470684             0.657977            0.926399   
 3               0.072643             0.479941            0.518759   
 4               1.056126            -0.594808            0.436537   
 
    Nuisance Feature 2  Class Y  
 0           -0.287491        0  
 1           -0.422267        0  
 2           -0.120003        0  
 3           -0.273960        0  
 4           -0.499097

In [199]:
df['Class Y'] = df['Class Y'].astype('str') 

In [200]:
X, y = df[['Informative Feature 1', 'Informative Feature 2',
        'Informative Feature 3', 'Redundant Feature 1', 'Nuisance Feature 1',
        'Nuisance Feature 2']].values, df['Class Y'].values

In [201]:
feature_names, class_names=['Informative Feature 1', 'Informative Feature 2',
        'Informative Feature 3', 'Redundant Feature 1', 'Nuisance Feature 1',
        'Nuisance Feature 2'],list(set(df['Class Y']))

In [202]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y ,shuffle=True)

In [203]:
from sklearn.linear_model import LinearRegression
lr_model = LogisticRegression()

In [204]:
lr_model.fit(X_train, y_train)

LogisticRegression()

In [205]:
y_preds = lr_model.predict(X_test)

In [206]:
import numpy as np
(y_preds==y_test).mean()

0.43333333333333335

In [216]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [217]:
# def create_explain_target_function(model, X_test, class_names, feature_names):
#     def get_explain_instance(comm, open_msg):
#         @comm.on_msg
#         def _recv(msg):
#             try:
#                 instance = msg['content']['data']['explain_instance']
#                 #explainer = explain_instance.lime['explainer']
#                 with open('lime_explainer','rb') as f:
#                     lime_explainer = dill.load(f)
                    
#                 with open('shap_explainer','rb') as f:
#                     shap_explainer = dill.load(f)
                    
#                 with open('shap_values','rb') as f:
#                     shap_values = dill.load(f)
                    
#                 with open('lime_values','rb') as f:
#                     lime_values = dill.load(f)
                    
#                 lime_res ={}
#                 shap_res={}
#                 for index, i in enumerate(class_names):
#                     lime_res[i] = lime_values[int(instance['instance_index'])]
#                     shap_res[i] = dict(zip(feature_names, list(shap_values[index][int(instance['instance_index']), :])))
                
#                 comm.send({'message':"LIME explanation obtained successfully!", "lime_explanation":lime_res, "shap_explanation": shap_res})
#             except Exception as e:
#                 exception_string  = str(e)
#                 comm.send({'message':"Error {0}\n{1}\n{2}".format(exception_string, msg, traceback.format_exc())})
#     return get_explain_instance

# get_ipython().kernel.comm_manager.register_target('get_explain_instance', create_explain_target_function(lr_model.predict_proba, X_test, class_names=dataset.target_names, feature_names=dataset.feature_names))

In [218]:
import traceback
import sys
import dill

def create_config_target_function(X_train, X_test, feature_names, class_names, model):
    explainer = None
    def get_configurations(comm, open_msg):
        @comm.on_msg
        def _recv(msg):
            try:
                print(msg)
                config = msg['content']['data']['config']
                discretize_continuous  = config['lime']['discretize_continuous']
                
                lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train,discretize_continuous = discretize_continuous , feature_names= feature_names, class_names = class_names )
                lime_values = []
                
                for i in range(len(X_test)):
                    i_explanation = {}
                    explanation = lime_explainer.explain_instance(X_test[i], lr_model.predict_proba, num_samples=100, top_labels = len(class_names))
                    for j, label in enumerate(class_names):
                        
                        j_explanation = explanation.as_list(j)
                        j_explanation = dict([[k[0], float(k[1])] for k in j_explanation])
                        i_explanation[label] = j_explanation
                    lime_values.append(i_explanation)
                    
                shap_explainer = shap.LinearExplainer(model, X_train, feature_dependence="independent")
                shap_values = shap_explainer.shap_values(X_test)
                shap_values = [i.tolist() for i in shap_values]
                with open('lime_explainer','wb') as f:
                    dill.dump(lime_explainer,f)
                    
                with open('shap_explainer','wb') as f:
                    dill.dump(shap_explainer,f)
                    
                with open('shap_values','wb') as f:
                    dill.dump(shap_values,f)
                    
                with open('lime_values','wb') as f:
                    dill.dump(lime_values,f)
                    
                #get_ipython().kernel.comm_manager.register_target('get_explain_instance', create_explain_target_function(model.predict_proba, X_test, class_names=dataset.target_names,feature_names=feature_names, lime_explainer=lime_explainer, shap_explainer=shap_explainer, shap_values=shap_values))
                
                comm.send({'message':"LIME and SHAP explainer created successfully!{0},{1}".format(discretize_continuous,config['lime']['discretize_continuous']), 
                        "class_names":list(lime_explainer.class_names),
                          'lime_values' : lime_values,
                          'shap_values' : shap_values,
                          'feature_names': feature_names,
                          'dataset': X_test.tolist()})
            except Exception as e:
                exception_string  = str(e)
                comm.send({'message':"Error {0}\n {1} \n {2}".format(exception_string, traceback.format_exc(), sys.exc_info()[2])})
            #comm.send({'message':'hi'})
    return get_configurations



In [219]:
get_ipython().kernel.comm_manager.register_target('get_configurations', create_config_target_function(X_train, X_test, feature_names, class_names, lr_model))

In [240]:
from string import Template
from IPython.display import display, HTML

js_string = Template("""
<html>
<head>
<script >


    
    
function main(){
  const lamp_1_div_id = "#lamp_graph_1_div"
  const lamp_2_div_id = "#lamp_graph_2_div"
  
  let comm_config = Jupyter.notebook.kernel.comm_manager.new_comm("get_configurations")
  let comm_explain = Jupyter.notebook.kernel.comm_manager.new_comm("get_explain_instance")
  let lime_explainer = null
  
  let lime_res_arrays = null
  let shap_res_arrays = null
  
  let class_names = null
  let dataset = null
  let lime_values = null
  let shap_values = null
  
  
  comm_config.on_msg(function(msg){
    res = msg.content.data
    console.log(res)
    class_names = res['class_names']
    feature_names = res['feature_names']
    lime_values = res['lime_values']
    shap_values = res['shap_values']
    dataset = res['dataset']
    update_class_dropdown(class_names)
    update_feature_dropdown(feature_names)
    
    var selected_class = document.getElementById("class").value
    var selected_feature = document.getElementById("feature").value

    create_lamp_1_graph(lamp_1_div_id, lime_values, shap_values, class_names, feature_names, selected_class)
    create_lamp_2_graph(lamp_2_div_id, lime_values, shap_values, class_names, feature_names, dataset, selected_class, selected_feature)

  })
  
  document.getElementById("configure").addEventListener("click", function(){
  
      config = {'lime':{}, 'shap':{}}
      //var discretize_continuous = document.getElementById("lime_discretize_continuous").checked
      var discretize_continuous = false
      config.lime.discretize_continuous = discretize_continuous
      comm_config.send({'config':config})
      
  })
  
  
  
    document.getElementById("class").addEventListener("click", function(){
    var selected_class = document.getElementById("class").value
    var selected_feature = document.getElementById("feature").value
    
    create_lamp_1_graph(lamp_1_div_id, lime_values, shap_values, class_names, feature_names, selected_class)
    create_lamp_2_graph(lamp_2_div_id, lime_values, shap_values, class_names, feature_names, dataset, selected_class, selected_feature)
    })
    
    document.getElementById("feature").addEventListener("click", function(){
    var selected_class = document.getElementById("class").value
    var selected_feature = document.getElementById("feature").value
    create_lamp_2_graph(lamp_2_div_id, lime_values, shap_values, class_names, feature_names, dataset, selected_class, selected_feature)
    })
  
}

function removeAll(selectBox) {
    while (selectBox.options.length > 0) {
        selectBox.remove(0);
    }
}

function update_class_dropdown(class_names){
    select_object = document.getElementById("class")
    removeAll(select_object)
    console.log(class_names)
    
    
    for(var i=0; i<class_names.length; i++){
        var option_object = document.createElement("option")
        var option_text = document.createTextNode(class_names[i])
        option_object.appendChild(option_text)
        option_object.setAttribute('value', class_names[i])
        select_object.appendChild(option_object)
    }
    
    
    console.log("DONE")
}

function update_feature_dropdown(feature_names){
    select_object = document.getElementById("feature")
    removeAll(select_object)
    
    for(var i=0; i<feature_names.length; i++){
        var option_object = document.createElement("option")
        var option_text = document.createTextNode(feature_names[i])
        option_object.appendChild(option_text)
        option_object.setAttribute('value', feature_names[i])
        select_object.appendChild(option_object)
    }
    
}





function create_lamp_2_graph(div_id, lime_values, shap_values, class_names, feature_names, dataset, selected_class, selected_feature){

    let chosen_class = class_names.indexOf(selected_class)
    let chosen_feature = feature_names.indexOf(selected_feature)
    
    let feature = d3.map(dataset, row => row[chosen_feature])
    
    let lime_chunk = d3.map(lime_values, row =>  row[class_names[chosen_class]][feature_names[chosen_feature]])
    
    let shap_chunk = d3.map(shap_values[chosen_class], row => row[chosen_feature])
    console.log(shap_chunk)
    console.log(feature)
    
    const lamp_graph_2 = ({
      'height' : 200,
      'width' : 300,
      'margins' : {
        'top' : 50,
        'bottom' : 50,
        'left' : 60,
        'right' : 60
      }})

    graph_div = document.getElementById(div_id.slice(1))
    graph_div.innerHTML = ''
    const svg = d3.select(div_id).append("svg")
    
    svg.attr('height', lamp_graph_2.height + lamp_graph_2.margins.top + lamp_graph_2.margins.bottom)
    svg.attr('width', lamp_graph_2.width + lamp_graph_2.margins.left + lamp_graph_2.margins.right)

    const graph = svg.append('g')
    .attr('transform', `translate(${lamp_graph_2.margins.left},${lamp_graph_2.margins.top})`)
    .attr('height', lamp_graph_2.height )
    .attr('width', lamp_graph_2.width )
    
    const colorScale = d3.scaleOrdinal()
    .domain(['lime', 'shap'])
    .range(d3.schemeCategory10)

    var legend = d3.legendColor()
    .scale(colorScale)
    .orient('vertical')
    .shapeWidth(20)

    svg.append('g')
    .attr('transform', `translate(${lamp_graph_2.width/2},10)`)
    .call(legend)
    
    const y1Scale = d3.scaleLinear()
    .domain(d3.extent(lime_chunk))
    .range([lamp_graph_2.height, 0]).nice()

    const y2Scale = d3.scaleLinear()
    .domain(d3.extent(shap_chunk))
    .range([lamp_graph_2.height, 0]).nice()
    
    const xScale = d3.scaleLinear()
    .domain(d3.extent(feature))
    .range([0, lamp_graph_2.width]).nice()

    const y1Axis = d3.axisLeft(y1Scale)
    const y2Axis = d3.axisRight(y2Scale)
    const xAxis = d3.axisBottom(xScale)
    
    lime_graph = graph.append('g')
    
    lime_graph.selectAll('circle')
    .data(Object.keys(lime_chunk))
    .join('circle')
    .attr('cy', row => y1Scale(lime_chunk[row]))
    .attr('cx', row => xScale(feature[row]))
    .attr('r', 2)
    .attr('fill', colorScale('lime'))
    
    shap_graph = graph.append('g')
    
    shap_graph.selectAll('circle')
    .data(Object.keys(shap_chunk))
    .join('circle')
    .attr('cy', row => y2Scale(shap_chunk[row]))
    .attr('cx', row => xScale(feature[row]))
    .attr('r', 2)
    .attr('fill', colorScale('shap'))


    graph.append('g')
    .call(y1Axis)
    .attr('stroke', 'black')
    .append('text')
    .attr('text-anchor', 'middle')
    .attr('transform', `translate(-45,${lamp_graph_2.height/2}) rotate(270)`)
    .attr('font-size', 12)
    .attr('stroke', 'black')
    .text('LIME Values')
    //return svg.node()
    
    graph.append('g')
    .attr('transform', `translate(${lamp_graph_2.width},0)`)
    .call(y2Axis)
    .attr('stroke', 'black')
    .append('text')
    .attr('text-anchor', 'middle')
    .attr('transform', `translate(40,${lamp_graph_2.height/2}) rotate(90)`)
    .attr('font-size', 12)
    .attr('stroke', 'black')
    .text('SHAP Values')
    //return svg.node()
    
    graph.append('g')
    .attr('transform', `translate(0,${lamp_graph_2.height})`)
    .call(xAxis)
    .attr('stroke', 'black')
    .append('text')
    .attr('text-anchor', 'middle')
    .attr('transform', `translate(${lamp_graph_2.width/2},40)`)
    .attr('font-size', 12)
    .attr('stroke', 'black')
    .text(selected_feature)
    //return svg.node()

    

}


function create_lamp_1_graph(div_id, lime_values, shap_values, class_names, feature_names, selected_class){


    let chosen_class = class_names.indexOf(selected_class)
    console.log(chosen_class)
    console.log(typeof(selected_class))
    console.log(class_names)
    lime_chunk = d3.map(lime_values, row => d3.map(feature_names, row2 => row[class_names[chosen_class]][row2]))
    console.log(lime_chunk)
    
    shap_chunk = shap_values[chosen_class]
    console.log(shap_chunk)
    
    graph_data = shap_chunk.keys()
    graph_data = d3.map(graph_data, row => d3.map(shap_chunk[row].keys(), row2 => [shap_chunk[row][row2], lime_chunk[row][row2]]))
    //graph_data = d3.map(graph_data, row => d3.sort(row, (a,b) => d3.ascending(a[0],b[0])))
    
    max_lime = d3.max(d3.map(lime_chunk, row => d3.max(row, a => a)), row=> row)
    min_lime = d3.min(d3.map(lime_chunk, row => d3.min(row, a => a)), row=> row)
    max_shap = d3.max(d3.map(shap_chunk, row => d3.max(row, a => a)), row=> row)
    min_shap = d3.min(d3.map(shap_chunk, row => d3.min(row, a => a)), row=> row)
    

    
  const lamp_graph_1 = ({
  'height' : 200,
  'width' : 400,
  'margins' : {
    'top' : 50,
    'bottom' : 50,
    'left' : 60,
    'right' : 10
  }})
  
  graph_div = document.getElementById(div_id.slice(1))
  graph_div.innerHTML = ''
  const svg = d3.select(div_id).append("svg")
  
  svg.attr('height', lamp_graph_1.height + lamp_graph_1.margins.top + lamp_graph_1.margins.bottom)
  svg.attr('width', lamp_graph_1.width + lamp_graph_1.margins.left + lamp_graph_1.margins.right)
  
  

   const graph = svg.append('g')
  .attr('transform', `translate(${lamp_graph_1.margins.left},${lamp_graph_1.margins.top})`)
  .attr('height', lamp_graph_1.height )
  .attr('width', lamp_graph_1.width )

    const colorScale = d3.scaleOrdinal()
    .domain(feature_names)
    .range(d3.schemeCategory10)

    var legend = d3.legendColor()
    .scale(colorScale)
    .orient('horizontal')
    .shapeWidth(20)
    .labels(d3.map(feature_names, row => {
    splits = row.split(" ")
    res = d3.map(splits, row2 => row2[0])
    return res.join("")
    }));

    svg.append('g')
    .attr('transform', `translate(${lamp_graph_1.width/2},10)`)
    .call(legend)

  const xScale = d3.scaleLinear()
  .domain([min_shap, max_shap])
  .range([0, lamp_graph_1.width]).nice()

  const yScale = d3.scaleLinear()
  .domain([min_lime,max_lime])
  .range([lamp_graph_1.height, 0]).nice()

  const xAxis = d3.axisBottom(xScale)
  const yAxis = d3.axisLeft(yScale)

  lines = graph.append('g')
  .selectAll('g')
  .data(graph_data.keys())
  .join('g')
  
  circles = graph.append('g')
  .selectAll('g')
  .data(graph_data.keys())
  .join('g')
  
  lines.selectAll('line')
  .data(row => d3.map(graph_data[row].keys(), row2 => [row, row2]))
  .join('line')
  .attr('x1', row => {

    if(row[1]==0){
    return xScale(graph_data[row[0]][row[1]][0])}
    else{return xScale(graph_data[row[0]][row[1]-1][0])}
  })
  .attr('y1',  row => {
  if(row[1]==0){return yScale(graph_data[row[0]][row[1]][1])}
    else{return yScale(graph_data[row[0]][row[1]-1][1])}
    })
  .attr('x2',  row => xScale(graph_data[row[0]][row[1]][0]))
  .attr('y2',  row => yScale(graph_data[row[0]][row[1]][1]))
  .attr('stroke-width', 1)
  .attr('opacity', 0.1)
  .attr('stroke', 'black')
  
  circles.selectAll('circle')
  .data(row => d3.map(graph_data[row].keys(), row2 => [row,row2]))
  .join('circle')
  .attr('cx', row => xScale(graph_data[row[0]][row[1]][0]))
  .attr('cy',  row => yScale(graph_data[row[0]][row[1]][1]))
  .attr('r', 3)
  .attr('opacity', 0.5)
  .attr('fill', row => colorScale(feature_names[row[1]]) )

  graph.append('g')
  .attr('transform', `translate(${0},${lamp_graph_1.height})`)
  .call(xAxis)
  .append('text')
  .attr('text-anchor', 'middle')
  .attr('transform', `translate(${lamp_graph_1.width/2},${30})`)
  .attr('font-size', 12)
  .attr('stroke', 'black')
  .text('SHAP Values')
  
  graph.append('g')
  .call(yAxis)
  .attr('stroke', 'black')
  .append('text')
  .attr('text-anchor', 'middle')
  .attr('transform', `translate(-40,${lamp_graph_1.height/2}) rotate(270)`)
  .attr('font-size', 12)
  .attr('stroke', 'black')
  .text('LIME Values')
  //return svg.node()
  
}

main()
</script>
</head>


<body>
<div id="parameters_div" lstyle="border:2px solid black;">


    <!-- <div id= "lime_config" style="display:inline-block;border:1px solid black; width:500px" > -->
    <div id= "lime_config" style="display:inline-block;border:1px solid black; width:500px" >
        <b>LIME Configurations:</b></br>
        <label for="lime_discritize_continuous">Descritise Continuous:</label>
        <label value=0 id ='lime_discretize_continuous' size=5>False</label></br>
        
        <label for "lime_n_samples">N Samples  :</label>
        <input  id ='lime_n_samples' size=5 value=100>
    </div>

    <div id= "shap_config" style="display:inline-block; border:1px solid black; width:470px" >
        <b>SHAP Configurations: </b></br>
        <label for="shap_feature_perturbation">Feature Perturbation</label>
        <select id ='shap_feature_perturbation'>
        <option value='interventional' name='Interventional'>Interventional</option>
        <option value='correlation_dependent' name='Correlation Dependent'>Correlation Dependent</option>        
        </select>
        </br>
        <label for "lime_n_samples">N Samples  :</label>
        <input  id ='lime_n_samples' size=5 value=100></br>
    </div>
    
    <center><input type="button" id="configure" value="Configure"> <br></center>

    <div id='output'> 
        Explanation for class: <select id='class'></select>
        Explanation for feature: <select id='feature'></select>
    </div>
    
    


</div>
</br>


<div id="lamp_graph_1_div" style="display:inline-block;width:470px; border:1px solid black;">
</div>

<div id="lamp_graph_2_div" style="display:inline-block;width:470px; border:1px solid black;">
</div>
</body>
</html>


""")

js_string = js_string.safe_substitute()
html_string = HTML(js_string)
_ = display(html_string)